<a href="https://colab.research.google.com/github/kalai2315/Langchain_for_AgentiAI/blob/main/Conversation_Chains_and_Memory__with_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U --quiet langchain langchain-google-genai langchain-community langchain-chroma


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  

In [2]:
import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google API Key: ")

Enter your Google API Key: ··········


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-latest")

In [4]:
from langchain_core.prompts import ChatPromptTemplate

In [5]:
prompt_txt = """{query}"""
prompt = ChatPromptTemplate.from_template(prompt_txt)

In [6]:
llm_chain = prompt | llm

In [7]:
response = llm_chain.invoke({"query": "What are the first four colours in rainbow?"})

In [8]:
print(response.content)

The first four colours in a rainbow are red, orange, yellow, and green.


In [9]:
response = llm_chain.invoke({"query": "and the other three?"})

In [10]:
print(response.content)

Please provide me with the context or information about the "other three" you're referring to.  I need more information to answer your question.


# Conversation chain with ConversationBufferMemory

In [11]:
#from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [12]:
# 1. System prompt
SYS_PROMPT = """ Act as a helpful assistant and give brief answers"""

# 2. Prompt with memory placeholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}")
    ]
)

# 3. Initialize memory
memory = ConversationBufferMemory(return_messages=True)

/tmp/ipython-input-12-1561153591.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [13]:
memory.load_memory_variables({})

{'history': []}

In [14]:
# 4. Get memory messages using the entire input dictionary
def get_memory_messages(inputs: dict) -> list:
    return memory.load_memory_variables(inputs)["history"]

get_memory_messages("what are the first four colours in rainbow?")

[]

In [15]:
RunnableLambda(get_memory_messages).invoke({"query": "what are the first four colours in rainbow?"})

[]

In [16]:
RunnablePassthrough.assign(
    history = RunnableLambda(get_memory_messages)
).invoke({"query": "what are the first four colours in rainbow?"}
)

{'query': 'what are the first four colours in rainbow?', 'history': []}

In [17]:
def get_memory_messages(query):
  return memory.load_memory_variables(query)["history"]

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    )
    | prompt
    | llm
)

In [19]:
query = {"query": "what are the first four colours in rainbow?"}
response = conversation_chain.invoke(query)
response

AIMessage(content='Red, orange, yellow, green.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--fb9dd154-0bfb-4ef6-9b16-c5e7d47cd81e-0', usage_metadata={'input_tokens': 18, 'output_tokens': 9, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [20]:
print(response.content)

Red, orange, yellow, green.


In [21]:
memory.load_memory_variables({})

{'history': []}

In [22]:
memory.save_context(query, {"output": response.content})

In [23]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='what are the first four colours in rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Red, orange, yellow, green.', additional_kwargs={}, response_metadata={})]}

In [24]:
query = {"query": "and the other three?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

Blue, indigo, violet.


In [25]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='what are the first four colours in rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Red, orange, yellow, green.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='and the other three?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Blue, indigo, violet.', additional_kwargs={}, response_metadata={})]}

In [26]:
query = {"query": "Explain AI in 3 bullet points?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

* **Mimics human intelligence:** AI systems are designed to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.
* **Learns from data:** AI algorithms improve their performance over time by analyzing vast amounts of data.
* **Diverse applications:** AI is used across many fields, from healthcare and finance to transportation and entertainment.


In [27]:
query = {"query": "Now do the same for Deep learning?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

* **Subset of AI:** Deep learning is a specialized type of machine learning, a subset of AI.
* **Artificial neural networks:** It uses artificial neural networks with multiple layers (hence "deep") to analyze data.
* **Feature extraction:**  Automatically learns complex features from raw data, eliminating the need for manual feature engineering.


In [28]:
query = {"query": "What have we discussed so far?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

We've discussed rainbows, AI, and deep learning.


In [29]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='what are the first four colours in rainbow?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Red, orange, yellow, green.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='and the other three?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Blue, indigo, violet.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Explain AI in 3 bullet points?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='* **Mimics human intelligence:** AI systems are designed to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.\n* **Learns from data:** AI algorithms improve their performance over time by analyzing vast amounts of data.\n* **Diverse applications:** AI is used across many fields, from healthcare and finance to transportation and entertainment.', additional_kwargs={}, response_metadata={}),
  HumanMessage(con

# Conversation chain with ConversationBufferWindowMemory

In [30]:
#from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [31]:
# 1. System prompt
SYS_PROMPT = """ Act as a helpful assistant and give brief answers"""

# 2. Prompt with memory placeholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}")
    ]
)

# 3. Initialize memory
memory = ConversationBufferWindowMemory(return_messages=True, k=2)

/tmp/ipython-input-31-720944166.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(return_messages=True, k=2)


In [32]:
def get_memory_messages(query):
  return memory.load_memory_variables(query)["history"]

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    )
    | prompt
    | llm
)

In [33]:
query = {"query": "what are the first four colours in rainbow?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

Red, orange, yellow, green.


In [34]:
query = {"query": "and the other three?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

Blue, indigo, violet.


In [35]:
query = {"query": "and the other three?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

There are only six colours in a rainbow.  There aren't "other three".


In [36]:
query = {"query": "Explain AI in 3 bullet points?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

* **Mimics human intelligence:** AI systems are designed to perform tasks that typically require human intelligence, like learning, problem-solving, and decision-making.
* **Learns from data:** AI algorithms improve their performance by analyzing vast amounts of data and identifying patterns.
* **Adapts and improves:**  Many AI systems can adapt to new information and refine their processes over time.


In [37]:
query = {"query": "Now do the same for Deep learning?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

* **Uses artificial neural networks:** Deep learning relies on complex, layered neural networks to analyze data.
* **Learns hierarchical features:** It automatically learns increasingly abstract features from raw data, unlike simpler machine learning models.
* **Requires large datasets:** Deep learning models need massive amounts of data to train effectively and achieve high accuracy.


In [38]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Explain AI in 3 bullet points?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='* **Mimics human intelligence:** AI systems are designed to perform tasks that typically require human intelligence, like learning, problem-solving, and decision-making.\n* **Learns from data:** AI algorithms improve their performance by analyzing vast amounts of data and identifying patterns.\n* **Adapts and improves:**  Many AI systems can adapt to new information and refine their processes over time.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Now do the same for Deep learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='* **Uses artificial neural networks:** Deep learning relies on complex, layered neural networks to analyze data.\n* **Learns hierarchical features:** It automatically learns increasingly abstract features from raw data, unlike simpler machine learning models.\n* **Requires large datas

In [39]:
query = {"query": "What have we discussed so far?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

We've discussed Artificial Intelligence (AI) generally and then focused specifically on Deep Learning, highlighting their key characteristics.


# Conversation chain with ConversationSummaryMemory

In [40]:
#from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


# 1. System prompt
SYS_PROMPT = """ Act as a helpful assistant and give brief answers"""

# 2. Prompt with memory placeholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}")
    ]
)

# 3. Initialize memory
memory = ConversationSummaryMemory(return_messages=True, llm=llm)


def get_memory_messages(query):
  return memory.load_memory_variables(query)["history"]

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    )
    | prompt
    | llm
)

/tmp/ipython-input-40-2398032088.py:20: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(return_messages=True, llm=llm)


In [41]:
query = {"query": "Explain AI in 3 bullet points?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

* **Mimicking human intelligence:** AI systems are designed to perform tasks that typically require human intelligence, like learning, problem-solving, and decision-making.

* **Learning from data:** AI algorithms learn from large datasets to improve their performance over time without explicit programming.

* **Applications across many fields:** AI is used in various sectors, including healthcare, finance, transportation, and entertainment.


In [42]:
query = {"query": "Now do the same for Deep learning?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

* Uses artificial neural networks with multiple layers.
* Learns complex patterns from massive datasets.
* Powers advanced applications like image recognition and natural language processing.


In [43]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human asks the AI to explain AI in three bullet points, which are that AI mimics human intelligence, learns from data to improve performance, and has applications across many fields.  The human then asks for a similar explanation of deep learning, which the AI describes as using artificial neural networks with multiple layers to learn complex patterns from massive datasets, powering applications like image recognition and natural language processing.', additional_kwargs={}, response_metadata={})]}

# Conversation chain with VectorStoreRetrieverMemory

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import VectorStoreRetrieverMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda


In [7]:
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


Create a vector database to store conversation history


In [8]:
chroma_db = Chroma(collection_name = "history_db",
                   embedding_function = embedding_model )

/tmp/ipython-input-8-2205453753.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(collection_name = "history_db",


In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import VectorStoreRetrieverMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


# 1. System prompt
SYS_PROMPT = """ Act as a helpful assistant and give brief answers"""

# 2. Prompt with memory placeholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{query}")
    ]
)

In [10]:
retriever = chroma_db.as_retriever(search_type = "similarity",
                                   search_kwargs = {"k": 2})
memory = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)


/tmp/ipython-input-10-2359598408.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=retriever, return_messages=True)


In [13]:
def get_memory_messages(query):
  return [memory.load_memory_variables(query)["history"]]

conversation_chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(get_memory_messages)
    )
    | prompt
    | llm
)

In [14]:
query = {"query": "Tell me about AI"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:1568: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


Artificial intelligence (AI) involves creating computer systems capable of performing tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.  It encompasses various techniques, including machine learning and deep learning.


In [15]:
query = {"query": "What about Deep Learning?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

Deep learning is a subset of machine learning that uses artificial neural networks with multiple layers (hence "deep") to analyze data and extract complex patterns.


In [16]:
query = {"query": "Tell me about the fastest animal in the world in two lines?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

The peregrine falcon is the fastest animal.  It can reach speeds exceeding 240 mph during its hunting stoop.


In [17]:
query = {"query": "What about cheetah?"}
response = conversation_chain.invoke(query)
memory.save_context(query, {"output": response.content})
print(response.content)

The cheetah is the fastest land animal, reaching speeds up to 75 mph in short bursts.


In [18]:
print(memory.load_memory_variables({"query": "What about Machine Learning?"})['history'])

query: What about Deep Learning?
output: Deep learning is a subset of machine learning that uses artificial neural networks with multiple layers (hence "deep") to analyze data and extract complex patterns.
query: Tell me about AI
output: Artificial intelligence (AI) involves creating computer systems capable of performing tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.  It encompasses various techniques, including machine learning and deep learning.


# MultiUser Conversation chains with ConversationMessageHistory

In [8]:
%pip install -U langchain


In [10]:
!pip show langchain

Name: langchain
Version: 0.3.26
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.memory import ConversationBufferMemory

In [12]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [16]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{human_input}")
])

In [31]:
history_store = {}

def get_session_history(session_id):
    if session_id not in history_store:
        history_store[session_id] = ConversationBufferMemory( memory_key="history",return_messages=True)
    return history_store[session_id].chat_memory

In [32]:
llm_chain = (prompt_template | llm)

In [33]:
#create conversation chain which can load memory based on specific user and session id
conversation_chain = RunnableWithMessageHistory(
    llm_chain,
    get_session_history=get_session_history,
    input_messages_key="human_input",
    history_messages_key="history"
)

In [34]:
def chat_with_llm(prompt:str, session_id:str):
  for chunk in conversation_chain.stream({"human_input": prompt}, config={"configurable": {"session_id": session_id}}):

    print(chunk.content, end="", flush=True)

In [35]:
#conversation chain for user1

user_id = 'ali123'
prompt = 'Hi, I am Ali, can you explain AI in 3 bullet points?'
chat_with_llm(prompt, user_id)

* **AI mimics human intelligence:**  AI systems are designed to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.

* **AI learns from data:** AI algorithms improve their performance over time by analyzing large amounts of data and identifying patterns.  This learning process allows them to adapt and become more accurate.

* **AI powers many applications:** AI is not just a single technology; it's a broad field encompassing many techniques used in various applications, from self-driving cars and medical diagnosis to personalized recommendations and language translation.


In [39]:
prompt = "Do the same for deep learning?"
chat_with_llm(prompt, user_id)

Let's explore deep learning, a powerful subset of machine learning.  Instead of explicitly programming rules, deep learning uses artificial neural networks with many layers ("deep") to learn from data.  This allows it to automatically learn complex patterns and representations from raw data, unlike simpler machine learning models that often require manual feature engineering.

Think of it like this:  imagine you're teaching a child to identify a cat.  With regular machine learning, you might tell the child to look for specific features like pointy ears, whiskers, and a furry tail.  With deep learning, you just show the child many pictures of cats and non-cats.  The deep learning model, like the child's brain, gradually learns to identify cats by itself, discovering subtle features and relationships between them that you might not even have consciously noticed.

Here's a breakdown:

* **Artificial Neural Networks (ANNs):**  Deep learning relies on ANNs, which are computing systems inspi

In [41]:
prompt = "Give a brief summary of what we have discussed?"
chat_with_llm(prompt, user_id)

We've covered the fundamentals of machine learning (ML) and deep learning (DL), two key areas of artificial intelligence.  ML involves algorithms that learn from data to make predictions without explicit programming, encompassing supervised, unsupervised, and reinforcement learning methods.  DL is a more advanced type of ML using deep artificial neural networks with many layers to automatically learn complex patterns from raw data, eliminating the need for manual feature engineering.  While powerful, DL requires significantly more data and computational resources than traditional ML.  We discussed the key differences between these approaches and their applications in various fields.


In [37]:
# Simulate second user
user_id = 'user_b456'
prompt = 'Hello, I am Bob. Can you explain what machine learning is?'

chat_with_llm(prompt, user_id)

Hi Bob!  Machine learning (ML) is a branch of artificial intelligence (AI) where computer systems learn from data without being explicitly programmed.  Instead of relying on hard-coded rules, ML algorithms identify patterns and relationships within data to make predictions or decisions.

Imagine you want to teach a dog a new trick.  You wouldn't write down a precise set of instructions; instead, you'd show the dog what to do, reward it for correct actions, and correct it when it's wrong.  Machine learning is similar. We feed the computer lots of data, and it learns to identify patterns and make predictions based on that data.

Here's a simplified breakdown:

* **Data:** This is the fuel for machine learning.  It could be anything from images and text to numbers and sensor readings.  The more relevant and high-quality data you have, the better the system will learn.
* **Algorithm:** This is the set of instructions that tells the computer how to learn from the data.  There are many diffe

In [38]:
prompt = "Do the same for deep learning?"
chat_with_llm(prompt, user_id)

Deep learning is a subfield of machine learning that uses artificial neural networks with multiple layers (hence "deep") to analyze data and extract higher-level features.  Think of it as a more sophisticated and powerful version of regular machine learning.

Here's a breakdown comparing it to regular machine learning:

**Regular Machine Learning:**

* **Feature Engineering:** Often requires manual feature extraction.  You need to explicitly tell the algorithm what features to look for in the data (e.g., for image recognition, you might tell it to look for edges, corners, and colors).
* **Simpler Models:** Uses simpler models with fewer parameters.
* **Less Data Required (Generally):** Can often achieve good results with relatively smaller datasets.
* **Faster Training (Generally):** Training is typically faster due to the simpler models.


**Deep Learning:**

* **Automatic Feature Extraction:** Automatically learns features from raw data without manual intervention.  For example, a de

In [40]:
prompt = "Give a brief summary of what we have discussed?"
chat_with_llm(prompt, user_id)

We've discussed the core concepts of machine learning and deep learning, highlighting their key differences and similarities.

**Machine learning** is a type of artificial intelligence where computer systems learn from data without explicit programming.  They identify patterns and relationships to make predictions or decisions.  We explored supervised learning (learning from labeled data), unsupervised learning (finding patterns in unlabeled data), and reinforcement learning (learning through trial and error).  Machine learning is used in many applications, from spam filtering to recommendation systems.

**Deep learning** is a specialized subset of machine learning that uses artificial neural networks with multiple layers to learn complex representations from raw data.  A key advantage is its ability to automatically learn features, eliminating the need for manual feature engineering required in many traditional machine learning approaches.  However, deep learning requires significantl

# Multi User Window Based Conversation chain - SQLChatMessageHistory

In [42]:
!rm memory.db #removes memory database file


rm: cannot remove 'memory.db': No such file or directory


In [54]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.memory.chat_message_histories import SQLChatMessageHistory
from langchain.memory import ChatMessageHistory
import os
from langchain_core.runnables import RunnablePassthrough, RunnableLambda


In [79]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///memory.db")

def get_session_history_db(session_id):
    return SQLChatMessageHistory(
        session_id=session_id,
        connection=engine
    )

In [80]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [81]:
#create memory buffer window function to return last k conversation

def memory_buffer_window(messages,k=2):
  return messages[-(k+1):]

In [82]:
llm_chain = (
    RunnablePassthrough.assign(history = lambda x: memory_buffer_window(x["history"])))|prompt|llm


In [87]:
#create conversation chain which can load memory based on specific user and session id
conversation_chain = RunnableWithMessageHistory(
    llm_chain,
    get_session_history_db,
    input_messages_key="input",
    history_messages_key="history",
    )

In [88]:
def chat_with_llm(prompt:str, session_id:str):
  for chunk in conversation_chain.stream({"input": prompt}, config={"configurable": {"session_id": session_id}}):

    print(chunk.content, end="", flush=True)

In [89]:
user_id = "boby123"
prompt ="which is the fastest animal?"
chat_with_llm(prompt, user_id)

The peregrine falcon is generally considered the fastest animal.  Its hunting stoop (a high-speed dive) can reach speeds exceeding 240 mph (386 km/h).


In [90]:
prompt ="which is the slowest animal?"
chat_with_llm(prompt, user_id)

There's no single definitive answer to what the slowest animal is, as "slowest" can depend on how you measure it (speed over a short distance, average speed over a lifetime, etc.). However, contenders for the title often include various species of **sea stars** and **giant tortoises**.  Their movements are incredibly slow compared to most other animals.


In [91]:
prompt ="which is the largest animal?"
chat_with_llm(prompt, user_id)

The largest animal is the **blue whale**.


In [92]:
prompt ="what topics have we discussed, show briefly as three bullet points?"
chat_with_llm(prompt, user_id)

* Slowest animal
* Largest animal
* Summary of our conversation topics
